### updateMII

This notebook inserts and updates data from an uploaded parquet file (`mii_incoming_file_path`) into a Master Incident Index (MII) delta table (`mii_delta_table_path`). 

In [ ]:
filename=""

In [ ]:
from pyspark.sql import SparkSession
from delta.tables import DeltaTable
from pyspark.sql.functions import col
from notebookutils import mssparkutils

spark = SparkSession.builder.getOrCreate()

# Set up file client
storage_account = "$STORAGE_ACCOUNT"
mii_incoming_file_path = f"abfss://patient-data@{storage_account}.dfs.core.windows.net/{filename}"
mii_delta_table_path = f"abfss://delta-tables@{storage_account}.dfs.core.windows.net/MII"


def update(mii_incoming_file_path,mii_delta_table_path):
    mii_updates = spark.read.parquet(mii_incoming_file_path)

    # Check if MII Delta table exists
    if DeltaTable.isDeltaTable(spark, mii_delta_table_path):
        # If the table exists, update records
        mii_main = DeltaTable.forPath(spark, mii_delta_table_path)

        mii_main.alias("mii_main") \
        .merge(
            mii_updates.alias("mii_updates"),
            "mii_updates.person_id = mii_main.person_id") \
        .whenMatchedUpdate(set ={
            "incident_id": "mii_updates.incident_id",
            "specimen_collection_date":"mii_updates.collection_date",
            "record_type":"mii_updates.record_type",
            "episode_date": "mii_updates.episode_date",
            "process_status": "mii_updates.process_status",
            "resolution_status": "mii_updates.resolution_status"}) \
        .whenNotMatchedInsert(values = { "person_id": col("mii_updates.person_id"),
        "incident_id": col("mii_updates.incident_id"),
        "specimen_collection_date": col("mii_updates.collection_date"),
        "record_type": col("mii_updates.record_type"),
        "episode_date": col("mii_updates.episode_date"),
        "process_status": col("mii_updates.process_status"),
        "resolution_status": col("mii_updates.resolution_status")}) \
        .execute()
    else:
        # If Delta table doesn't exist, create it.
        mii_updates.write.format("delta").mode("append").save(mii_delta_table_path)

    

    
update(mii_incoming_file_path,mii_delta_table_path)

In [ ]:
# Move file that triggered the MII update event into the archive folder
destination = f"abfss://patient-data@{storage_account}.dfs.core.windows.net/archive/{filename}"
mssparkutils.fs.mv(src=mii_incoming_file_path,dest=destination,create_path=True)